In [141]:
import gurobipy as gp
from gurobipy import GRB

# Index and Parameter

In [142]:
# Index sets
P = [...]                 # all patients
E = [...]                 # HaH-eligible patients
H = [...]                 # high-risk patients (must stay in hospital)
K = [...]                 # condition groups / bundle types
J = [...]                 # depots / hospitals
T = [...]                 # planning horizon, e.g. range(14) for 14 days

# Patient parameters
l_hosp = {i: ... for i in P}                 # predicted inpatient LOS
l_home = {i: ... for i in P}                 # predicted HaH LOS
demand = {(i, t): ... for i in E for t in T} # daily bundle demand d_it (0 if t >= l_home[i])

# Cost & capacity parameters
B      = ...                        # hospital bed capacity
c_hosp = ...                        # daily inpatient cost
c_s    = ...                        # daily in-home nursing cost
c_p    = {k: ... for k in K}        # procurement cost per bundle type k
c_v    = ...                        # inventory holding cost per bundle per day
c_d    = ...                        # per-kilometre delivery cost
alpha  = ...                        # TSP routing coefficient (≈ 0.7–1.2)
M_big  = 1e6                        # Big-M for linking z and a

y0 = {(k, j): 0 for k in K for j in J}

# Example Index

In [143]:
# Index sets
P = [0, 1, 2, 3]          # 4 patients
E = [0, 1, 2]             # eligible for HaH
H = [3]                   # must stay in hospital
K = [0, 1]                # two bundle / condition groups
J = [0]                   # single depot / hospital
T = range(5)              # 5-day planning horizon (t = 0 - 4)

# Length of stay predictions (days)
l_hosp = {0: 5, 1: 4, 2: 6, 3: 7}
l_home = {0: 4, 1: 3, 2: 5, 3: 0}   

# Daily bundle demand d_it  (only for HaH-eligible patients E)
# format: demand[(patient, day)] = bundles_needed
demand = {(i, t): 0 for i in E for t in T}  # init zeros
# patient 0 needs 1 bundle/day for 4 days
for t in range(4):
    demand[(0, t)] = 1
# patient 1 needs 1 bundle/day for 3 days
for t in range(3):
    demand[(1, t)] = 1
# patient 2 needs 1 bundle/day for 5 days (but horizon is only 5)
for t in range(5):
    demand[(2, t)] = 1

# Capacity & cost parameters
B      = 2        
c_hosp = 120.0   
c_s    =  250.0  
c_p    = {0: 95.0, 1: 120.0}  
c_v    =   3.0    
c_d    =   2.2   
alpha  =   0.8    
M_big  = 1e6      

# Initial inventory (start with zero stock)
y0 = {(k, j): 0.0 for k in K for j in J}


In [144]:
import random
from collections import defaultdict

# =========== CONFIG =========== #
NUM_PATIENTS   = 100          # total patients
ELIGIBLE_RATIO = 0.85         # % HaH-eligible
K              = [0, 1, 2]    # 3 bundle / condition groups
J              = [0]          # one depot
HOSP_MIN, HOSP_MAX = 3, 8     # inpatient LOS range
HOME_MIN, HOME_MAX = 5, 12    # HaH LOS range (> hosp)
CONST_RATIO, LHL_RATIO, HL_RATIO = 0.4, 0.3, 0.3   # demand-curve mix
SEED = 42
# ============================== #

random.seed(SEED)

# ----- 1. Index sets -----
P = list(range(NUM_PATIENTS))
E = random.sample(P, int(NUM_PATIENTS * ELIGIBLE_RATIO))
H = [i for i in P if i not in E]
T = range(HOME_MAX)           # 0 … 11 covers最长 HaH



# ----- 2. LOS dictionaries -----
l_hosp, l_home = {}, {}
for i in P:
    l_hosp[i] = random.randint(HOSP_MIN, HOSP_MAX)
    min_home  = max(HOME_MIN, l_hosp[i] + 2)   # ensure longer at home
    l_home[i] = random.randint(min_home, HOME_MAX) if i in E else 0

# ----- 3. Demand (i,t) -----
patterns = random.choices(
    ["CONST", "LHL", "HL"],
    weights=[CONST_RATIO, LHL_RATIO, HL_RATIO],
    k=len(E)
)

demand = defaultdict(int)            # default 0
for i, pat in zip(E, patterns):
    L = l_home[i]
    if pat == "CONST":               # constant 1 bundle/day
        for t in range(L): demand[(i, t)] = 1
    elif pat == "LHL":               # low-high-low shape
        mid = L // 2
        for t in range(L):
            if t < mid * 0.5 or t >= L - mid * 0.5:
                demand[(i, t)] = 1
            else:
                demand[(i, t)] = 2
    else:                            # HL: high then taper
        half = L // 2
        for t in range(L):
            demand[(i, t)] = 2 if t < half else 1

# ----- 4. Cost & capacity -----
B      = 42          # beds (example)
c_hosp = 300.0
c_s    = 250.0
c_p    = {0: 95.0, 1: 120.0, 2: 110.0}
c_v    = 3.0
c_d    = 2.2
alpha  = 100
M_big  = 1e6

# ----- 5. Initial inventory -----
y0 = {(k, j): 0.0 for k in K for j in J}

# -------- Quick sanity print --------
print("Patients:", len(P), " |  HaH-eligible:", len(E), " |  Horizon days:", len(T))
print("Sample LOS (first 5):", [(i, l_hosp[i], l_home[i]) for i in P[:5]])
print("Demand curve mix:", {p: patterns.count(p) for p in set(patterns)})


Patients: 100  |  HaH-eligible: 85  |  Horizon days: 12
Sample LOS (first 5): [(0, 6, 11), (1, 5, 0), (2, 8, 12), (3, 7, 10), (4, 8, 11)]
Demand curve mix: {'LHL': 28, 'HL': 21, 'CONST': 36}


# DV & Obj.

In [145]:
m = gp.Model('HaH_joint_selection_supply')

# DV
x = m.addVars(P, vtype=GRB.BINARY,      name='x')            # 1 = HaH, 0 = stay in hospital
y = m.addVars(K, J, T, lb=0,            name='y')            # inventory level
z = m.addVars(E, K, J, T, lb=0,         name='z')            # quantity delivered
a = m.addVars(E, J, T, vtype=GRB.BINARY, name='a')           # delivery indicator

N = m.addVars(J, T, lb=0,   name='N')           # number of patients served
w = m.addVars(J, T, lb=0,                name='w')           # sqrt(N)
L = m.addVars(J, T, lb=0,                name='L')           # route length (km)

# Objective
in_hosp_cost   = gp.quicksum(c_hosp * l_hosp[i] * (1 - x[i]) for i in P)
nurse_cost     = gp.quicksum(c_s    * l_home[i] * x[i]       for i in E)
procure_cost   = gp.quicksum(c_p[k] * z[i, k, j, t]          for i in E for k in K for j in J for t in T)
transport_cost = gp.quicksum(c_d    * L[j, t]               for j in J for t in T)
holding_cost   = gp.quicksum(c_v    * y[k, j, t]            for k in K for j in J for t in T)

m.setObjective(in_hosp_cost + nurse_cost +
               procure_cost + transport_cost + holding_cost,
               GRB.MINIMIZE)

# Constraints

In [146]:
# High-risk Ineligibility
for i in H:
    m.addConstr(x[i] == 0)

# Bed Capacity
m.addConstr(gp.quicksum(1 - x[i] for i in P) <= B)

# Demand Fulfillment for HaH Patients
for i in E:
    for t in T:
        m.addConstr(gp.quicksum(z[i,k,j,t] for k in K for j in J) ==
                    demand[(i,t)] * x[i]
                    )


# Inventory
for k in K:
    for j in J:
        # Day 0
        m.addConstr(
            y[k, j, 0] ==
            y0[k, j] + gp.quicksum(z[i, k, j, 0] for i in E)
        )
        # Day 1 and on
        for t in T[1:]:
            m.addConstr(
                y[k, j, t] ==
                y[k, j, t - 1] + gp.quicksum(z[i, k, j, t] for i in E)
            )

# Link quantity delivered to delivery indicator
for i in E:
    for j in J:
        for t in T:
            m.addConstr(
                gp.quicksum(z[i, k, j, t] for k in K) <= M_big * a[i, j, t]
            )

# Count number of deliveries per depot/day
for j in J:
    for t in T:
        m.addConstr(
            N[j, t] == gp.quicksum(a[i, j, t] for i in E)
        )

# TSP
for j in J:
    for t in T:
        m.addGenConstrPow(N[j,t], w[j,t], 0.5, name=f"sqrt_{j}_{t}")  # w = √N
        m.addConstr(alpha * w[j,t] <= L[j,t])


In [147]:
m.Params.TimeLimit = 600 
m.optimize()

if m.SolCount:
    print(f"\nTotal cost = {m.objVal:.2f}")
    selected_HaH = [i for i in E if x[i].X > 0.5]
    print("HaH patient IDs:", selected_HaH)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 165H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 2116 rows, 4252 columns and 12263 nonzeros
Model fingerprint: 0x214e319f
Model has 12 function constraints treated as nonlinear
  12 POW
Variable types: 3132 continuous, 1120 integer (1120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+01, 6e+01]
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 165H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 2116 r

In [148]:
print("hosp  :", in_hosp_cost.getValue())
print("nurse :", nurse_cost.getValue())
print("proc  :", procure_cost.getValue())
print("trans :", transport_cost.getValue())
print("hold  :", holding_cost.getValue())

hosp  : 69900.0
nurse : 129250.0
proc  : 59470.0
trans : 16681.07723599527
hold  : 15042.0
